In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
from skimpy import skim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pdb
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
np.random.seed(42)
# import pandas_profiling

In [ ]:
# subdirs = os.listdir('/Users/awxlong/Desktop/my-studies/temp_data/CRC/TCGA-CRC/gdc_download_20240629_132916.799980/')
cohort_name = 'COAD'
task_name = 'g0_arrest'
# Parent directory containing subdirectories with XML files
# parent_directory = '/Users/awxlong/Desktop/my-studies/temp_data/CRC/TCGA-CRC/gdc_download_20240629_132916.799980/'
root_dir = '/Users/awxlong/Desktop/my-studies/hpc_exps/Data/'

df = pd.read_csv(f'{root_dir}{cohort_name}_WSI_clinical_g0_features.csv')


In [ ]:
df['Ann Arbor'].value_counts()

In [ ]:
df['Residual Tumor'].value_counts()
# df['Residual Tumor'].unique()

In [ ]:
df['Person Neoplasm Cancer Status'].value_counts()

In [ ]:
print(df.shape)
df.head()

In [ ]:
df['g0_arrest'].value_counts()

In [ ]:
df['Gender'].value_counts()

In [ ]:
df['Race'].value_counts()

In [ ]:
df['Other Dx'].value_counts()

In [ ]:
df['Pathologic Stage'].value_counts()

In [ ]:
df['Icd 10'].value_counts()

### icd codes

In [ ]:
def replace_icd_o_3_site(df, column_name):
    icd_o_3_site_map = {
        'C18.0': 'Cecum',
        'C18.2': 'Ascending colon',
        'C18.3': 'Hepatic flexure of colon',
        'C18.4': 'Transverse colon',
        'C18.5': 'Splenic flexure of colon',
        'C18.6': 'Descending colon',
        'C18.7': 'Sigmoid colon',
        'C18.9': 'Colon, NOS (Not Otherwise Specified)',
        'C19.9': 'Rectosigmoid junction',
        'C20.9': 'Rectum, NOS',
        'C49.4': 'Connective and soft tissue of abdomen',
        'C80.9': 'Unknown primary site'
    }
    
    # Replace the codes with their meanings
    df[column_name] = df[column_name].map(icd_o_3_site_map).fillna(df[column_name])
    
    # Replace any remaining codes with 'Unknown code'
    # df[column_name] = df[column_name].apply(lambda x: 'Unknown code' if x.startswith('C') else x)
    
    return df

def replace_icd_o_3_histology(df, column_name):
    icd_o_3_histology_map = {
        '8140/3': 'Adenocarcinoma, NOS',
        '8480/3': 'Mucinous adenocarcinoma',
        '8260/3': 'Papillary adenocarcinoma, NOS',
        '8560/3': 'Adenosquamous carcinoma',
        '8574/3': 'Adenocarcinoma with neuroendocrine differentiation',
        '8255/3': 'Adenocarcinoma with mixed subtypes',
        '8010/3': 'Carcinoma, NOS',
        '8263/3': 'Adenocarcinoma in tubulovillous adenoma',
        '8211/3': 'Tubular adenocarcinoma'
    }
    
    # Replace the codes with their meanings
    df[column_name] = df[column_name].map(icd_o_3_histology_map).fillna(df[column_name])
    
    # Replace any remaining codes with 'Unknown histology'
    # df[column_name] = df[column_name].apply(lambda x: 'Unknown histology' if isinstance(x, str) and '/' in x else x)
    
    return df

def replace_icd_10(df, column_name):
    icd_10_map = {
        'C18.0': 'Malignant neoplasm of cecum',
        'C18.2': 'Malignant neoplasm of ascending colon',
        'C18.3': 'Malignant neoplasm of hepatic flexure',
        'C18.4': 'Malignant neoplasm of transverse colon',
        'C18.5': 'Malignant neoplasm of splenic flexure',
        'C18.6': 'Malignant neoplasm of descending colon',
        'C18.7': 'Malignant neoplasm of sigmoid colon',
        'C18.9': 'Malignant neoplasm of colon, unspecified',
        'C19': 'Malignant neoplasm of rectosigmoid junction',
        'C20': 'Malignant neoplasm of rectum',
        'C49.4': 'Malignant neoplasm of connective and soft tissue of abdomen',
        'C80.1': 'Malignant (primary) neoplasm, unspecified'
    }
    
    # Replace the codes with their meanings
    df[column_name] = df[column_name].map(icd_10_map).fillna(df[column_name])
    
    # Replace any remaining codes with 'Unknown ICD-10 code'
    df[column_name] = df[column_name].apply(lambda x: 'Unknown ICD-10 code' if isinstance(x, str) and x.startswith('C') else x)
    
    return df

df = replace_icd_o_3_site(df, 'Icd O 3 Site')
df = replace_icd_o_3_histology(df, 'Icd O 3 Histology')
df = replace_icd_10(df, 'Icd 10')

In [ ]:
df['Icd O 3 Histology'].value_counts()

In [ ]:
df['Anatomic Neoplasm Subdivision'].value_counts()


In [ ]:
df['Icd O 3 Site'].value_counts()

In [ ]:
# df['Icd 10'].value_counts()

In [ ]:
# df['Gleason Grading'].value_counts()
df['Preoperative Pretreatment Cea Level'].describe()

In [ ]:
skim(df)

In [ ]:
# # Numerical features
# num_feats=[col for col in df.columns if df[col].dtypes != 'object' and col != 'Unnamed: 0']

# # Plot distribution of numerical columns
# fig=plt.figure(figsize=(20,30))
# for i, col in enumerate(num_feats):
#     plt.subplot(len(num_feats),1,1*i+1)
#     sns.distplot(df[col])
    
# fig.tight_layout()
# plt.show()

# Numerical features
num_feats = [col for col in df.columns if df[col].dtypes != 'object' and col != 'Unnamed: 0']

# Calculate the number of rows and columns for the subplot grid
n = len(num_feats)
cols = 3
rows = math.ceil(n / cols)

# Create a larger figure with more space between subplots
fig, axes = plt.subplots(rows, cols, figsize=(20, 5*rows))
fig.suptitle('Distribution of Numerical Variables', fontsize=24, y=1.02)

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Increase the space between subplots
plt.subplots_adjust(hspace=0.5, wspace=0.3)

for i, col in enumerate(num_feats):
    ax = axes[i]
    
    # Plot histogram and KDE
    sns.histplot(data=df, x=col, kde=True, ax=ax)
    
    # Add mean and median lines
    mean = df[col].mean()
    median = df[col].median()
    ax.axvline(mean, color='r', linestyle='--', label=f'Mean: {mean:.2f}')
    ax.axvline(median, color='g', linestyle='-.', label=f'Median: {median:.2f}')
    
    # Customize the plot
    ax.set_title(col, fontsize=16)
    ax.set_xlabel(col, fontsize=14)
    ax.set_ylabel('Count', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.legend(fontsize=12)

# Remove any unused subplots
for i in range(n, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()


In [ ]:
print(df.columns)

In [ ]:
categorical_vars = [col for col in df.columns if df[col].dtypes == 'object' and col != 'PatientID'] + ['g0_arrest']


In [ ]:
for col in categorical_vars:
    plt.figure(figsize=(8, 6))
    counts = df[col].value_counts(normalize=True)
    
    # Get a list of unique colors for the bars
    colors = plt.cm.get_cmap('tab20', len(counts))(np.linspace(0, 1, len(counts)))
    
    # Plot the bars with labels
    counts.plot(kind='bar', color=colors)
    plt.xticks(rotation=90)
    plt.xlabel(col)
    plt.ylabel('Proportion')
    plt.title(f'Distribution of {col}')
    
    # Add labels to the bars
    for i, v in enumerate(counts):
        plt.text(i, v, f"{v:.2f}", ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()


In [ ]:
var_interest = 'Icd 10'
# Assuming your DataFrame is called 'df'
g0_arrest_counts = df.groupby([var_interest, 'g0_arrest']).size().unstack(fill_value=0)
g0_arrest_proportions = g0_arrest_counts.div(g0_arrest_counts.sum(axis=1), axis=0)

fig, ax = plt.subplots(figsize=(10, 6))
g0_arrest_proportions.plot(kind='bar', ax=ax)

ax.set_xlabel('Gender')
ax.set_ylabel('Proportion')
ax.set_title('Proportion of g0_arrest by Gender')
ax.legend(title='g0_arrest')

plt.tight_layout()
plt.show()


In [ ]:
var_interest = 'Icd 10'
# Assuming your DataFrame is called 'df'
g0_arrest_counts = df.groupby([var_interest, 'g0_arrest']).size().unstack(fill_value=0)
g0_arrest_proportions = g0_arrest_counts.div(g0_arrest_counts.sum(axis=1), axis=0)

fig, ax = plt.subplots(figsize=(10, 6))

# 1) Make the bars stacked
g0_arrest_proportions.plot(kind='bar', stacked=True, ax=ax)

# 2) Label each bar with the proportion
bottom = 0
for i in range(len(g0_arrest_proportions.columns)):
    for j in range(len(g0_arrest_proportions)):
        proportion = g0_arrest_proportions.iloc[j, i]
        # ax.text(j, bottom + proportion / 2, f"{proportion:.2f}", ha='center', va='center', color='white')
        x = j  # Use the row index as the x-coordinate
        y = proportion  # Use the proportion value as the y-coordinate
        ax.text(x, y, f"{proportion:.2f}", ha='center', va='bottom', color='black')
    bottom += g0_arrest_proportions.iloc[:, i]

ax.set_xlabel(var_interest)
ax.set_ylabel('Proportion')
ax.set_title('Proportion of g0_arrest by Icd 10')
ax.legend(title='g0_arrest')

plt.tight_layout()
plt.show()

### mode impute missing values

In [ ]:
def imputation(split, train_split=df):
   mode_values = train_split.mode().iloc[0]

   imputed_split = split.fillna(mode_values)

   return imputed_split



print("before imputation:", sum(df.isna().sum()))
# print("Number of NaN values per feature: ")
# df_train_na = df_train.apply(mode_imputation, axis = 1)
df = imputation(df)
print("after imputation:", sum(df.isna().sum()))

### convert from object to category

In [ ]:
def cast_category(split, train_cat_vars=categorical_vars):
    split[train_cat_vars] = split[train_cat_vars].astype('category')
    # split = pd.get_dummies(split, columns=train_cat_vars, prefix= train_cat_vars)
    return split
# df_train[categorical_vars] = df_train[categorical_vars].astype('category')
df = cast_category(df)

In [ ]:
df.dtypes

### one hot encode categorical variables

In [ ]:
def one_hot_encode(split):
  cats = [col for col in split.columns if split[col].dtypes == 'category' and col != 'readmitted']
  split = pd.get_dummies(split, columns=cats, prefix= cats)
  return split

df = one_hot_encode(df)

In [ ]:
# df['Circumferential Resection Margin'].unique()


In [ ]:
print(df.shape)
df.head()